# Cats vs Dogs - CNN

Oldjuk meg ugyanazt a problémát konvolúciós neurális hálózatokkal!

Első körben a képek mérete legyen a korábban használt $48 \times 48$, így jól összehasonlítható lesz a megoldás a korábbival. A *batch_size* értékét nyugodtan magasabbra állíthatjuk amennyiben GPUt is használunk a tanításhoz: be fog férni a memóriájába.

In [7]:
from keras.preprocessing.image import ImageDataGenerator

imsize = 48

train_gen = ImageDataGenerator(rescale=1./255, 
                               featurewise_std_normalization=True)

train_data = train_gen.flow_from_directory("catsvsdogs\\training_set",
                                           target_size=(imsize, imsize),
                                           class_mode='binary',
                                           #batch_size=16
                                           batch_size=128
                                          )

test_gen = ImageDataGenerator(rescale=1./255, featurewise_std_normalization=True)

test_data = test_gen.flow_from_directory("catsvsdogs\\test_set",
                                           target_size=(imsize, imsize),
                                           class_mode='binary',
                                           #batch_size=16
                                           batch_size=128
                                         )

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


A felépítés a korábban használt Conv -- Pool rétegpárokból lesz felépítve, a konvolúciós kernel méretet egyre csökkentve, a filterek számát pedig növelve. A pontosság a Pooling rétegek kivételével növelhető lehet.

Az overfitting problémájával célszerű már most foglalkozni: a Dropout regularizáció kiváló megoldást jelent konvolúciós filterek esetén!

In [8]:
param_num = imsize*imsize*3

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(imsize, imsize, 3)))
model.add(Dropout(0.25))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Dropout(0.25))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(Dropout(0.25))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(400, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(400, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              #optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 44, 44, 32)        2432      
                                                                 
 dropout_4 (Dropout)         (None, 44, 44, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 22, 22, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 20, 20, 64)        18496     
                                                                 
 dropout_5 (Dropout)         (None, 20, 20, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                  

Érdemes megfigyelni, hogy a modell paraméterszáma az előző mintához képest elenyésző.

In [10]:
from keras.callbacks import TensorBoard

tbcb = TensorBoard(log_dir="./logs/CNN-imsize48-adam")

In [11]:
model.fit_generator(train_data,
                    epochs=20,
                    validation_data=test_data,
                    callbacks=[tbcb]
                   )

C:\Users\dezsics\AppData\Local\Temp\ipykernel_6780\1109840961.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,


FailedPreconditionError: {{function_node __wrapped__CreateSummaryFileWriter_device_/job:localhost/replica:0/task:0/device:CPU:0}} . is not a directory [Op:CreateSummaryFileWriter]

A modell az előző, nem konvolúciós struktúrához képest jelentős mértékben javult:
- a tanítás folyamata sokkal rövidebb, egy epoch lényegesen rövidebb ideig tart
- a konvergencia sokkal meredekebb görbét mutat: gyakorlatilag néhány epoch alatt $70%$ feletti a tanítási pontosság, gyakorlatilag $~20$ epoch után $90%$ feletti a tanítási pontosság. Persze a validációs adathalmazon mért pontosság a konkrét mérőszám, de a *training accuracy* meredek emelkedése nagyszerűen jelzi hogy a modell tanul.

Ez utóbbi pedig azt jelenti hogy inkább az overfitting problémájával kell foglalkozni, mint a nagy paraméterszámmal és lassú konvergenciával.